In [ ]:
import requests
import xml.etree.ElementTree as ET
import time

In [ ]:
#define variables for queries
zenodobaseurl = "https://zenodo.org/oai2d?verb=ListRecords&"
initialquery = "metadataPrefix=oai_datacite3&set=user-zenodo"
resumptiontokenquery = "resumptionToken="
results = requests.get(zenodobaseurl+initialquery)

In [ ]:
root = ET.fromstring(results.content)
tree = ET.ElementTree(root)

In [ ]:
listofrecords = root.findall('{http://www.openarchives.org/OAI/2.0/}ListRecords')[0]

In [ ]:
#gets a resumption token
#makes resumption token False if there isn't one
def getresumptiontoken(theroot):
    global resumptiontoken
    for child in theroot.findall('{http://www.openarchives.org/OAI/2.0/}ListRecords'):
        if child.findall('{http://www.openarchives.org/OAI/2.0/}resumptionToken'):
            for grandchild in child.findall('{http://www.openarchives.org/OAI/2.0/}resumptionToken'):
                resumptiontoken = grandchild.text
        else:
            resumptiontoken = False
    #return resumptiontoken

In [ ]:
getresumptiontoken(root)

In [ ]:
print(resumptiontoken)

In [ ]:
#uses the resumption token to get more results
#new results are appended to the current results in listofrecords
#continues to loop until there is no longer a resumption token
while resumptiontoken:
    newresults = requests.get(zenodobaseurl+resumptiontokenquery+resumptiontoken)
    newroot = ET.fromstring(newresults.content)
    for child in newroot.findall('{http://www.openarchives.org/OAI/2.0/}ListRecords'):
        for grandchild in child.iter('{http://www.openarchives.org/OAI/2.0/}record'):
            listofrecords.append(grandchild)
    getresumptiontoken(newroot)
    print(resumptiontoken)
    time.sleep(2.5)

In [ ]:
tree.write('zenodoresults4-7-2015.xml')